<a href="https://colab.research.google.com/github/jeeda4/ML/blob/master/ML_JAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jeeda AbuKhader 

Machine Learning- Loading Data and Classification 


In [0]:
#setting up  
import functools
import numpy as np
import tensorflow as tf

In [3]:
#Importing the data 
Train_Data = "https://raw.githubusercontent.com/jeeda4/ML/master/train.csv"
Test_Data = "https://raw.githubusercontent.com/jeeda4/ML/master/test.csv"

train_file_path = tf.keras.utils.get_file("train.csv", Train_Data)
test_file_path = tf.keras.utils.get_file("test.csv", Test_Data)


1499136/1492816 [==============================] - 0s 0us/step


I split the data in R into train data and test data and put it on github. I use the link from my github in order to import the data. 

In [4]:
#Load Data
!head {train_file_path}

"","X","dvcat","weight","dead","airbag","seatbelt","frontal","sex","ageOFocc","yearacc","yearVeh","abcat","occRole","deploy","injSeverity","caseid"
"1",1,"25-39",25.0689999999013,"alive","none","belted",1,"f",26,1997,1990,"unavail","driver",0,3,"2:3:1"
"3",3,"10-24",32.3789999997243,"alive","none","none",1,"f",69,1997,1988,"unavail","driver",0,4,"2:5:1"
"4",4,"25-39",495.443999998271,"alive","airbag","belted",1,"f",53,1997,1995,"deploy","driver",1,1,"2:10:1"
"5",5,"25-39",25.0689999999013,"alive","none","belted",1,"f",32,1997,1988,"unavail","driver",0,3,"2:11:1"
"6",6,"40-54",25.0689999999013,"alive","none","belted",1,"f",22,1997,1985,"unavail","driver",0,3,"2:11:2"
"11",11,"10-24",812.868999995291,"alive","none","belted",1,"m",21,1997,1987,"unavail","pass",0,0,"2:14:2"
"13",13,"10-24",923.995999999344,"alive","airbag","belted",1,"m",67,1997,1991,"deploy","driver",1,1,"2:16:2"
"16",16,"25-39",29.0809999997728,"alive","none","belted",1,"m",71,1997,1990,"unavail","driver",0,1,"2:18:2"
"1

In [0]:
#Specify col to be classified
LABEL_COLUMN = 'injSeverity'
LABELS = [0, 1, 2, 3, 4, 5, 6]

In [0]:
#read csv from file 
#create dataset
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      select_columns = ['dvcat', 'weight', 'dead', 'airbag','seatbelt','frontal','sex','ageOFocc','yearacc','yearVeh','abcat','occRole','deploy','injSeverity'],
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [8]:
show_batch(raw_train_data)

dvcat               : [b'10-24' b'25-39' b'40-54' b'1-9km/h' b'1-9km/h']
weight              : [335.014 114.079  54.303 916.902 197.853]
dead                : [b'alive' b'alive' b'alive' b'alive' b'alive']
airbag              : [b'airbag' b'none' b'airbag' b'airbag' b'none']
seatbelt            : [b'belted' b'belted' b'none' b'none' b'none']
frontal             : [1 1 1 0 0]
sex                 : [b'f' b'm' b'm' b'm' b'm']
ageOFocc            : [55 38 22 24 44]
yearacc             : [1999 1999 2001 1997 1998]
yearVeh             : [1996 1989 1997 1997 1988]
abcat               : [b'nodeploy' b'unavail' b'deploy' b'nodeploy' b'unavail']
occRole             : [b'driver' b'driver' b'pass' b'driver' b'driver']
deploy              : [0 0 1 0 0]


Data Processing: 


In [0]:
#data processing 

#for continious data 

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features
    return features, labels



In [0]:
NUMERIC_FEATURES = ['weight' , 'frontal' , 'ageOFocc' , 'yearacc', 'yearVeh', 'deploy']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))


In [11]:
show_batch(packed_train_data)

dvcat               : [b'10-24' b'25-39' b'25-39' b'25-39' b'10-24']
dead                : [b'alive' b'alive' b'alive' b'alive' b'alive']
airbag              : [b'airbag' b'none' b'airbag' b'none' b'none']
seatbelt            : [b'belted' b'belted' b'belted' b'belted' b'none']
sex                 : [b'f' b'f' b'm' b'f' b'm']
abcat               : [b'deploy' b'unavail' b'nodeploy' b'unavail' b'unavail']
occRole             : [b'pass' b'driver' b'driver' b'driver' b'driver']
numeric             : [[2.55585e+02 1.00000e+00 5.10000e+01 1.99800e+03 1.99500e+03 1.00000e+00]
 [8.39340e+01 0.00000e+00 2.60000e+01 1.99900e+03 1.99200e+03 0.00000e+00]
 [5.80100e+02 0.00000e+00 2.40000e+01 2.00000e+03 1.99500e+03 0.00000e+00]
 [4.59502e+02 1.00000e+00 8.00000e+01 1.99900e+03 1.98900e+03 0.00000e+00]
 [7.77506e+02 1.00000e+00 1.60000e+01 1.99800e+03 1.98500e+03 0.00000e+00]]


In [0]:
#normalizing continious data
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc
example_batch, labels_batch = next(iter(packed_train_data)) 

In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

In [15]:
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(6,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f36936a1730>, mean=array([4.56156649e+02, 6.38645206e-01, 3.72780532e+01, 1.99955534e+03,
       1.99281684e+03, 3.35342131e-01]), std=array([1.40915921e+03, 4.80411400e-01, 1.80156145e+01, 1.70271235e+00,
       5.57710976e+00, 4.72127938e-01])))

In [16]:
example_batch['numeric']

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[3.752800e+01, 0.000000e+00, 2.000000e+01, 1.997000e+03,
        1.997000e+03, 1.000000e+00],
       [2.817200e+01, 1.000000e+00, 7.300000e+01, 1.999000e+03,
        1.993000e+03, 1.000000e+00],
       [2.614400e+01, 1.000000e+00, 5.000000e+01, 2.000000e+03,
        1.983000e+03, 0.000000e+00],
       [7.270900e+01, 0.000000e+00, 3.800000e+01, 2.000000e+03,
        1.997000e+03, 1.000000e+00],
       [3.017681e+03, 1.000000e+00, 3.800000e+01, 2.001000e+03,
        1.995000e+03, 0.000000e+00]], dtype=float32)>

In [17]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.2970769 , -1.3293715 , -0.95905983, -1.500722  ,  0.750049  ,
         1.4077917 ],
       [-0.30371633,  0.7521777 ,  1.9828324 , -0.32612547,  0.03283161,
         1.4077917 ],
       [-0.3051555 ,  0.7521777 ,  0.7061622 ,  0.2611728 , -1.7602118 ,
        -0.7102781 ],
       [-0.27211094, -1.3293715 ,  0.04007339,  0.2611728 ,  0.750049  ,
         1.4077917 ],
       [ 1.8177677 ,  0.7521777 ,  0.04007339,  0.8484711 ,  0.3914403 ,
        -0.7102781 ]], dtype=float32)

In [0]:
CATEGORIES = {
    'dvcat': ['1-9km/h', '10-24', '25-39', '40-54', '55+'],
    'dead' : ['alive', 'dead'],
    'airbag' : ['airbag', 'none'],
    'seatbelt' : ['belted', 'none'],
    'sex' : ['f', 'm'],
    'abcat' : ['deploy', 'nodeploy', 'unavail'],
    'occRole' : ['driver', 'pass']
}

In [19]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='dvcat', vocabulary_list=('1-9km/h', '10-24', '25-39', '40-54', '55+'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='dead', vocabulary_list=('alive', 'dead'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='airbag', vocabulary_list=('airbag', 'none'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='seatbelt', vocabulary_list=('belted', 'none'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('f', 'm'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='abcat', vocabulary_list=('deploy', 'nodeplo

In [20]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0.]


In [21]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.          0.          0.          1.          0.          1.
  0.          0.          0.          1.          0.          0.
 -0.2970769  -1.3293715  -0.95905983 -1.500722    0.750049    1.4077917
  1.          0.          1.          0.          1.          0.        ]



# Building the model

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

#model.fit(train_data, epochs=2)

Train, evaluate, and predict:

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [24]:
model.fit(train_data, epochs=20)

Epoch 1/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 2/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 3/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 4/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 5/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 6/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 7/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 8/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 9/20
2608/2608 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2144
Epoch 10/20
2608/2608 [=======================

In [25]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

2606/2606 [==============================] - 4s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2150


Test Loss 0.0, Test Accuracy 0.2149547040462494


Our model's accuracy rate is 21.5%. That is extremley low
 


In [28]:
predictions = model.predict(test_data)
print(predictions)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


Confusion Matrix:

In [0]:
#from tensorflow.contrib.metrics import streaming_accuracy
#y_ = LABELS 
#y = predictions
#confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=7)